# The Purpose of this code section is to add false negatives to our training data sample for klimaBERT


In [6]:
import pandas as pd

First run of KlimaBERT on the votes data. Finding false negatives in the sample to feed them back to the algorithm as new training data.

In [2]:
climate_votes = pd.read_pickle("/home/gugy/Documents/NLP-Climate-Politics-DK/climate_classifier/scored_data/votes_data_climate.pkl")
no_climate_votes = pd.read_pickle("/home/gugy/Documents/NLP-Climate-Politics-DK/climate_classifier/scored_data/votes_data_non_climate.pkl")


In [3]:
no_climate_votes.dropna(inplace=True)
climate_votes.drop_duplicates(subset='description_text', inplace=True)
climate_votes.reset_index(drop=True, inplace=True)


Generally good quality data for the climate votes (True Positives) --> Can be inspected below

In [4]:
climate_votes['date']
climate_votes['description_text'][15]

"Beslutningsforslaget opfordrede finansministeren til at trække aktstykke nr. 53 om salg af aktier i DONG Energy A/S tilbage. Ifølge forslagsstillerne var der flere gode grunde til at revurdere det fremlagte aktstykke, bl.a. et usædvanligt forslag om, at ledelsen, banker og rådgivere i forbindelse med salget fritages for ansvar. Endvidere var der en række verserende sager mod DONG fra kunder og andre aktionærer i DONG, og fra en klimapolitisk synsvinkel var det efter forslagsstillernes opfattelse forkert at privatisere landets største energiproducent. En aktuel sag med DONG Energy's byggeri af et kulkraftværk i Tyskland viste de negative klimapolitiske konsekvenser ved at drive selskabet efter økonomiske og ikke klimamæssige hensyn."

Looking for false negatives to add as new training data for klimaBERT

In [5]:
import re

klimaforandringer = re.compile(r"(K|k)lima(forandringer)?(ne)?")
co2 = re.compile(r"(CO 2)|(CO2)|(kulstof(udledning(en)?)?)|(co2)|drivhusgas(ser)?|klimaplan")
grøn_omstilling = re.compile(r"grøn(ne)? omstilling")
global_opvarmning = re.compile(r"global(e)? opvarmning")



First inspection with some regex queries

In [6]:
no_climate_votes['klimaforandringer'] = no_climate_votes['description_text'].apply(lambda x: 1 if re.search(klimaforandringer, x)!=None else 0)
no_climate_votes['CO2'] = no_climate_votes['description_text'].apply(lambda x: 1 if re.search(co2, x)!=None else 0)
no_climate_votes['grøn_omstilling'] = no_climate_votes['description_text'].apply(lambda x: 1 if re.search(grøn_omstilling, x)!=None else 0)
no_climate_votes['global_opvarmning'] = no_climate_votes['description_text'].apply(lambda x: 1 if re.search(global_opvarmning, x)!=None else 0)


There seem to be some duplicates (Worth looking into some data_cleaning here perhaps --> alternative drop the duplicates)

In [7]:
p_false_negatives = no_climate_votes[(no_climate_votes.CO2 == 1) | (no_climate_votes.klimaforandringer==1) | (no_climate_votes.grøn_omstilling==1)]

p_false_negatives['description_text']

#There seem to be some duplicates (Worth looking into some data_cleaning here)
p_false_negatives.drop_duplicates(subset='description_text', inplace=True)
p_false_negatives.reset_index(inplace=True, drop=True)


/home/gugy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
print(len(p_false_negatives))
p_false_negatives['false_negative'] = 0
p_false_negatives['description_text'][57]

58


/home/gugy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


'Formålet med lovforslaget er at fremme udnyttelsen af overskudsvarme. Lovforslaget gennemfører initiativer, der har ophæng i Energiaftale af 29. juni 2018, Aftale om øget udnyttelse af overskudsvarme af 23. marts 2019, Klimaaftale for energi og industri mv. af 22. juni 2020  (S, V, DF, RV, SF, EL, KF, LA og ALT) og Opfølgende aftale om fremme af overskudsvarme af 7. september 2021 mellem samme partier. Lovforslaget indeholder tre hovedpunkter; prisregulering af overskudsvarme, herunder en bagatelgrænse for prisregulering, en energieffektiviseringsordning for overskudsvarme og afgiftsfritagelse for energieffektiv ekstern overskudsvarme.'

In [12]:
index_lst = [0, 1, 2, 6, 9, 13, 17, 18, 20, 26, 52, 53, 56, 57]

for i in index_lst:
    p_false_negatives.at[i, 'false_negative'] = 1
p_false_negatives[['description_text', 'false_negative']]

len(p_false_negatives[p_false_negatives['false_negative']==1])

view_false_negatives = p_false_negatives[p_false_negatives['false_negative']==1]

In [18]:
view_false_negatives['label'] = 1 
view_false_negatives = view_false_negatives[['description_text', 'label']]


/home/gugy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
view_true_negatives = no_climate_votes.sample(n=14, random_state=30)
view_true_negatives['label'] = 0

view_true_negatives = view_true_negatives[['description_text', 'label']]

In [32]:
frames = [view_true_negatives, view_false_negatives]

votes_to_add = pd.concat(frames)

votes_to_add.to_pickle("votes_false_true_negatives.pkl")


### Improvements based on the meetings data

The following added data is based on the negatives from a first prediction iteration using KlimaBert V2 (with twitter_data and folketings data)

In [39]:
non_climate_meeting_quotes = pd.read_csv('ft_meetings_non_climate_50k_1.csv')

In [40]:
import re

#Some Danish climate-related regex:
#klima = '(K|k)limaet)'
klima_forandringer = '(K|k)lima(forandringer)(ne)?|global(e)? opvarmning|(K|k)limakrise(n)|klimabelastende?'
co2 = 'CO 2|CO2|CO_2|Co_2|CO2|Kuldioxid|drivhusgas(udledning(en)?)?|kulstofsudledningen'
målsætning = 'grøn(ne)? omstilling|CO 2-reduction|CO 2-neutral'
politik = 'Paris-aftalen|COP\d{1,2}|CO 2-afgift|CO 2-skat'
løsninger = 'klimaløsning|CO 2-nedsættende'

complete_klima_regex = re.compile("(%s|%s|%s|%s)" % (klima_forandringer, co2, målsætning, politik))

complete_klima_regex

re.compile(r'((K|k)lima(forandringer)(ne)?|global(e)? opvarmning|(K|k)limakrise(n)|klimabelastende?|CO 2|CO2|CO_2|Co_2|CO2|Kuldioxid|drivhusgas(udledning(en)?)?|kulstofsudledningen|grøn(ne)? omstilling|CO 2-reduction|CO 2-neutral|Paris-aftalen|COP\d{1,2}|CO 2-afgift|CO 2-skat)',
re.UNICODE)

In [44]:
non_climate_meeting_quotes['climate_regex'] = non_climate_meeting_quotes['text'].apply(lambda x: 1 if re.search(complete_klima_regex, x)!=None else 0)

In [49]:
view_quotes = non_climate_meeting_quotes[non_climate_meeting_quotes['climate_regex'] == 1]
view_quotes.reset_index(drop=True, inplace=True)

In [292]:
view_quotes['text'][3]

#One key finding here is that many of the false negatives are the very long texts

"Jeg vil gerne starte med at nævne det lille jubilæum, som dagen i dag faktisk også markerer, for vi har jo 1-års-jubilæum her i Folketinget nu i dag. Jeg ved godt, at det ikke altid har været lige nemt. Jeg tror også, at der er mange, for hvem det er overraskende, at vi ville komme op på et helt år, og der er også delte meninger i salen om det. Jeg vil gerne være den første til at ønske tillykke med, at der her ved åbningsdebatten er gået præcis 1 år, siden vi sidste gang havde fornøjelsen af at have hr. Lars Løkke Rasmussen heroppe på talerstolen til at holde en tale. Det er 1 år siden, Venstres formand havde noget politik, som han mente han kunne stå på mål for her på landets vigtigste talerstol. Dengang tog han ordet kl. 23.57 som privatist, hedder det, og det er måske meget sigende, altså det her med privatist – dermed mener jeg alle de millionergaver i form af overbetaling af privathospitaler, som hr. Lars Løkke Rasmussen stod for som sundhedsminister; opblomstringen af private s

In [97]:
import nltk

sent_breakdown = {'text': [],
}

#Change this to get a different quote sample (random vs. cliamte regexed)
full_quotes = view_quotes  #full_quotes_climate_sample

for index, row in full_quotes.iterrows():
    
    full_stops = re.compile("\.|\?|\!")
    sents = re.split(full_stops, row.text)
    
    number_of_sentences = 6
    
    previous_index = 0
    for i in range(0,len(sents), number_of_sentences):
        sent_breakdown['text'].append(".".join(sents[previous_index:i]).strip()+".")
        
        previous_index = i
        if i + 2 >= len(sents):
            sent_breakdown['text'][-1] += ".".join(sents[i:-1])
        elif i + 3 >= len(sents):
            sent_breakdown['text'].append(".".join(sents[i:-1]).strip()+".")
    

In [98]:
#Actually 6 sents!
three_sent_breakdown_view = pd.DataFrame(sent_breakdown)

In [99]:
three_sent_breakdown_view['climate_regex'] = three_sent_breakdown_view['text'].apply(lambda x: 1 if re.search(complete_klima_regex, x)!=None else 0)

In [132]:
six_sent_view = three_sent_breakdown_view[three_sent_breakdown_view['climate_regex']==1].reset_index(drop=True)


In [134]:
i = 10

In [339]:
i += 1
print(i)
print(six_sent_view['text'][i])

183
Tak for det, formand. Jeg vil gerne takke Enhedslisten for at have taget initiativ til den her debat om, hvilke konkrete initiativer regeringen vil tage for at opfylde sin egen målsætning om at reducere udledningen af CO 2 -udslippet med 40 pct. i 2020. Det er jo ikke noget nyt forslag. Både før valget og i regeringsgrundlaget så vi jo det her helt konkrete forslag om at reducere CO 2 -udledningen i 2020 med 40 pct. Man kan vel godt på en dag som denne sådan lidt frækt sige, at det her jo er et af de få valgløfter, som regeringen har i sinde at gennemføre.


In [340]:
#climate related texts from the the false negatives sample (when broken up)
index_lst2 = [0, 4, 5, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 33, 34, 37, 38, 44, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 78, 81, 84, 85, 86, 87, 92, 93, 94, 100, 103, 106, 107, 110, 111, 114, 115, 
              117, 118, 130, 142, 143, 144, 163, 172, 179, 182]
len(index_lst2)


65

In [13]:
positives = 229 + 65 + 14
print(positives)
negatives = 14 + 474
print(negatives)
print("diff", negatives-positives)

total = negatives + positives
print(total)
balance = positives / total
print(balance)


308
488
diff 180
796
0.3869346733668342


In [ ]:
for i in index_lst2:
    six_sent_view.at[i, 'label'] = 1

six_sent_false_negatives = six_sent_view[six_sent_view.label == 1.0]
six_sent_false_negatives['label'] =six_sent_false_negatives['label'].apply(lambda x: int(x))


In [350]:

six_sent_false_negatives[['text', 'label']].to_pickle("50k_ft_meetings_false_negatives.pkl")

### Inspecting the originally labeled data

In [7]:
original_labeled_data = pd.read_csv("/home/gugy/Documents/NLP-Climate-Politics-DK/climate_classifier/labeling/800_3sent_label703.csv")

In [8]:
original_labeled = original_labeled_data[['text', 'label']][0:703]

In [313]:
original_labeled['label'] = original_labeled['label'].apply(lambda x: int(x))

In [9]:
original_labeled[original_labeled['label'] == 0]

,text,label
0,"Det her gør vi med åbent sind, med løftet pand...",0.0
1,"For hvordan kan vi fejre demokratiet, når det ...",0.0
2,"Børn, som vokser op i sådanne områder, får ind...",0.0
4,"Men jeg vil blot helt overordnet sige, at rege...",0.0
5,"I Nye Borgerlige foreslog vi, at dagens beslut...",0.0
...,...,...
697,For eksempel var der en diskussion før mellem ...,0.0
698,"I en situation, hvor den Venstreledede regerin...",0.0
700,Og lige så vel som på andre dele af natur- og ...,0.0
701,"Vi kunne ikke være med, fordi den indeholder f...",0.0
